# Pipeline prata: Câmara dos Deputados

Este notebook extrai os dados da fase bronze e popula uma base de dados DuckDB com estes normalizados e deduplicados (ainda sem regularização de chaves estrangeiras)

# 1. Preparação

## 1.1 Imports

In [88]:
import os
import duckdb
%load_ext sql

from dotenv import load_dotenv

from pathlib import Path

load_dotenv()

DB_PATH = Path(os.getenv("SILVER_DUCKDB_PATH", ""))
DB_PATH.parent.mkdir(parents=True, exist_ok=True)
TMP_DIR = "/tmp/duckdb_tmp"  # make sure it exists

conn = duckdb.connect(DB_PATH)

conn.execute(f"SET temp_directory='{TMP_DIR}'")
conn.execute("SET memory_limit='64GB'")
conn.execute("SET threads=1")
conn.execute("SET preserve_insertion_order=false")

%sql conn --alias duck

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## 1.2 Macros

In [89]:
%%sql

CREATE OR REPLACE MACRO jget1(j, p) AS json_extract_string(j, p);

Running query in 'duck'

Count


# 2. Extração inicial

## 2.1. Modelos dimensionais

### 2.1.1 Bloco

In [90]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_bloco AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/senado/bloco/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'bloco';

SELECT COUNT(*) AS n FROM bronze_senado_bloco;


Running query in 'duck'

n
6


In [91]:
%%sql
SELECT * FROM bronze_senado_bloco;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
senado,bloco,335,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""335"",""DataCriacao"":""2023-02-01"",""Membros"":{""Membro"":[{""DataAdesao"":""2023-04-25"",""DataDesligamento"":""2024-01-30"",""Partido"":{""CodigoPartido"":""565"",""NomePartido"":""Rede Sustentabilidade"",""SiglaPartido"":""REDE""}},{""DataAdesao"":""2023-02-01"",""Partido"":{""CodigoPartido"":""63"",""NomePartido"":""Partido Socialista Brasileiro"",""SiglaPartido"":""PSB""}},{""DataAdesao"":""2023-02-01"",""DataDesligamento"":""2023-02-01"",""Partido"":{""CodigoPartido"":""66"",""NomePartido"":""Partido da Social Democracia Brasileira"",""SiglaPartido"":""PSDB""}},{""DataAdesao"":""2023-02-01"",""DataDesligamento"":""2025-02-17"",""Partido"":{""CodigoPartido"":""71"",""NomePartido"":""Partido dos Trabalhadores"",""SiglaPartido"":""PT""}},{""DataAdesao"":""2023-02-01"",""Partido"":{""CodigoPartido"":""557"",""NomePartido"":""Partido Social Democrático"",""SiglaPartido"":""PSD""}},{""DataAdesao"":""2023-02-01"",""DataDesligamento"":""2023-02-01"",""Partido"":{""CodigoPartido"":""565"",""NomePartido"":""Rede Sustentabilidade"",""SiglaPartido"":""REDE""}}]},""NomeApelido"":""BLPRD"",""NomeBloco"":""Bloco Parlamentar da Resistência Democrática""}",17bb2bcc628caccad8d592ed0ae509d8d6aff8e9d03f04324ca84857ad41349f,2023
senado,bloco,337,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""337"",""DataCriacao"":""2023-02-03"",""NomeApelido"":""Minoria"",""NomeBloco"":""Minoria""}",e05b037fcdb0ba14965f881c4b889e3536a9830d1ecd9a4b4526f8fbc283b44f,2023
senado,bloco,346,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""346"",""DataCriacao"":""2023-03-20"",""Membros"":{""Membro"":[{""DataAdesao"":""2023-03-20"",""Partido"":{""CodigoPartido"":""418"",""NomePartido"":""Progressistas"",""SiglaPartido"":""PP""}},{""DataAdesao"":""2023-03-20"",""Partido"":{""CodigoPartido"":""521"",""NomePartido"":""REPUBLICANOS"",""SiglaPartido"":""REPUBLICANOS""}}]},""NomeApelido"":""BLALIANÇA"",""NomeBloco"":""Bloco Parlamentar Aliança""}",0bfad6ac5099f84aef8d5d7b5cf7e2c7de9343b10e14bd5c9ad76171efba426a,2023
senado,bloco,357,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""357"",""DataCriacao"":""2025-02-03"",""Membros"":{""Membro"":[{""DataAdesao"":""2025-02-03"",""Partido"":{""CodigoPartido"":""523"",""NomePartido"":""Partido Liberal"",""SiglaPartido"":""PL""}},{""DataAdesao"":""2025-02-03"",""Partido"":{""CodigoPartido"":""566"",""NomePartido"":""Partido Novo"",""SiglaPartido"":""NOVO""}}]},""NomeApelido"":""BLVANG"",""NomeBloco"":""Bloco Parlamentar Vanguarda""}",329482b8ae5ed25112eeece8b3425cde3cee1531352b728789a3fe6c16b8e09a,2025
senado,bloco,358,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""358"",""DataCriacao"":""2025-02-18"",""Membros"":{""Membro"":[{""DataAdesao"":""2025-02-18"",""Partido"":{""CodigoPartido"":""582"",""NomePartido"":""União Brasil"",""SiglaPartido"":""UNIÃO""}},{""DataAdesao"":""2025-02-18"",""Partido"":{""CodigoPartido"":""577"",""NomePartido"":""Podemos"",""SiglaPartido"":""PODEMOS""}},{""DataAdesao"":""2025-02-18"",""Partido"":{""CodigoPartido"":""33"",""NomePartido"":""Movimento Democrático Brasileiro"",""SiglaPartido"":""MDB""}},{""DataAdesao"":""2025-02-18"",""Partido"":{""CodigoPartido"":""66"",""NomePartido"":""Partido da Social Democracia Brasileira"",""SiglaPartido"":""PSDB""}}]},""NomeApelido"":""BLDEM"",""NomeBloco"":""Bloco Parlamentar Democracia""}",7a8f88b1a99dcd2e88fdeb67839b2ceb62ce1883ac778de28c5960c8d0890930,2025
senado,bloco,359,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""359"",""DataCriacao"":""2025-02-18"",""Membros"":{""Membro"":[{""DataAdesao"":""2025-02-18"",""Partido"":{""CodigoPartido"":""20"",""NomePartido"":""Part

In [92]:

%%sql
DROP TABLE IF EXISTS bloco_senado;
CREATE TABLE bloco_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_senado_bloco
)
SELECT
    -- CodigoBloco
    CAST(jget1(j, '$.CodigoBloco') AS BIGINT) AS codigo_bloco,
    -- DataCriacao
    CAST(jget1(j, '$.DataCriacao') AS DATE) AS data_criacao,
    -- NomeApelido
    jget1(j, '$.NomeApelido') AS nome_apelido,
    -- NomeBloco
    jget1(j, '$.NomeBloco') AS nome_bloco,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.CodigoBloco') IS NOT NULL;
DROP VIEW IF EXISTS bronze_senado_bloco;


Running query in 'duck'

Success


### 2.1.2 Colegiado

In [93]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_colegiado AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/senado/colegiado/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'colegiado';

SELECT COUNT(*) AS n FROM bronze_senado_colegiado;

Running query in 'duck'

n
18


In [94]:
%%sql
DROP TABLE IF EXISTS colegiado_senado;
CREATE TABLE colegiado_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_senado_colegiado
)
SELECT
    -- CodigoColegiado
    CAST(jget1(j, '$.CodigoColegiado') AS BIGINT) AS codigo_colegiado,
    -- CodigoTipoColegiado
    CAST(jget1(j, '$.CodigoTipoColegiado') AS BIGINT) AS codigo_tipo_colegiado,
    -- DataInicio
    CAST(jget1(j, '$.DataInicio') AS DATE) AS data_inicio,
    -- IndicadorDistrPartidaria
    jget1(j, '$.IndicadorDistrPartidaria') AS indicador_distr_partidaria,
    -- NomeColegiado
    jget1(j, '$.NomeColegiado') AS nome_colegiado,
    -- SiglaColegiado
    jget1(j, '$.SiglaColegiado') AS sigla_colegiado,
    -- ordem
    CAST(jget1(j, '$.ordem') AS INTEGER) AS ordem,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.CodigoColegiado') IS NOT NULL;

DROP TABLE IF EXISTS tipo_colegiado_senado;
CREATE TABLE tipo_colegiado_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year
    FROM bronze_senado_colegiado
),
typed AS (
    SELECT
        CAST(jget1(j, '$.tipocolegiado.CodigoNaturezaColegiado') AS BIGINT) AS codigo_natureza_colegiado,
        CAST(jget1(j, '$.tipocolegiado.CodigoTipoColegiado') AS BIGINT) AS codigo_tipo_colegiado,
        jget1(j, '$.tipocolegiado.DescricaoTipoColegiado') AS descricao_tipo_colegiado,
        jget1(j, '$.tipocolegiado.IndicadorAtivo') AS indicador_ativo,
        jget1(j, '$.tipocolegiado.SiglaCasa') AS sigla_casa,
        jget1(j, '$.tipocolegiado.SiglaTipoColegiado') AS sigla_tipo_colegiado,
        year AS year_snapshot
    FROM base
    WHERE jget1(j, '$.tipocolegiado.CodigoTipoColegiado') IS NOT NULL
)
SELECT
    codigo_tipo_colegiado,
    arg_max(codigo_natureza_colegiado, year_snapshot) AS codigo_natureza_colegiado,
    arg_max(descricao_tipo_colegiado,  year_snapshot) AS descricao_tipo_colegiado,
    arg_max(indicador_ativo,           year_snapshot) AS indicador_ativo,
    arg_max(sigla_casa,                year_snapshot) AS sigla_casa,
    arg_max(sigla_tipo_colegiado,      year_snapshot) AS sigla_tipo_colegiado,
    max(year_snapshot)                              AS year_snapshot
FROM typed
GROUP BY codigo_tipo_colegiado;



DROP VIEW IF EXISTS bronze_senado_bloco;

Running query in 'duck'

Success


### 2.1.3 Parlamentar

In [95]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_parlamentar AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/senado/parlamentar/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'parlamentar';

SELECT COUNT(*) AS n FROM bronze_senado_parlamentar;

Running query in 'duck'

n
1298


In [96]:
%%sql
DROP TABLE IF EXISTS parlamentar_senado;

CREATE TABLE parlamentar_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year
    FROM bronze_senado_parlamentar
),
par_root AS (
    SELECT
        json_extract(j, '$.DetalheParlamentar.Parlamentar') AS par,
        year
    FROM base
),
flattened AS (
    SELECT
        -- Identificação
        CAST(jget1(par, '$.IdentificacaoParlamentar.CodigoParlamentar') AS BIGINT) AS codigo_parlamentar,
        CAST(jget1(par, '$.IdentificacaoParlamentar.CodigoPublicoNaLegAtual') AS BIGINT) AS codigo_publico_leg_atual,
        jget1(par, '$.IdentificacaoParlamentar.NomeCompletoParlamentar') AS nome_completo,
        jget1(par, '$.IdentificacaoParlamentar.NomeParlamentar') AS nome_parlamentar,
        jget1(par, '$.IdentificacaoParlamentar.SexoParlamentar') AS sexo_parlamentar,
        jget1(par, '$.IdentificacaoParlamentar.SiglaPartidoParlamentar') AS sigla_partido,
        jget1(par, '$.IdentificacaoParlamentar.UfParlamentar') AS uf_parlamentar,
        jget1(par, '$.IdentificacaoParlamentar.EmailParlamentar') AS email_parlamentar,
        -- Dados básicos
        CAST(jget1(par, '$.DadosBasicosParlamentar.DataNascimento') AS DATE) AS data_nascimento,
        jget1(par, '$.DadosBasicosParlamentar.EnderecoParlamentar') AS endereco_parlamentar,
        jget1(par, '$.DadosBasicosParlamentar.Naturalidade') AS naturalidade,
        jget1(par, '$.DadosBasicosParlamentar.UfNaturalidade') AS uf_naturalidade,
        year AS year_snapshot
    FROM par_root
)
SELECT *
FROM flattened
WHERE codigo_parlamentar IS NOT NULL;

DROP VIEW IF EXISTS bronze_senado_parlamentar;


Running query in 'duck'

Success


### 2.1.4 Partido

In [97]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_partido AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/senado/partido/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'partido';

SELECT COUNT(*) AS n FROM bronze_senado_partido;

Running query in 'duck'

n
103


In [98]:

%%sql
DROP TABLE IF EXISTS partido_senado;
CREATE TABLE partido_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_senado_partido
)
SELECT
    CAST(jget1(j, '$.Codigo') AS BIGINT) AS codigo_partido,
    CAST(jget1(j, '$.DataCriacao') AS DATE) AS data_criacao,
    jget1(j, '$.Nome') AS nome,
    jget1(j, '$.Sigla') AS sigla,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.Codigo') IS NOT NULL;
DROP VIEW IF EXISTS bronze_senado_partido;

Running query in 'duck'

Success


### 2.1.5 Processo

In [99]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_processo AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/senado/processo/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'processo';

SELECT COUNT(*) AS n FROM bronze_senado_processo;

Running query in 'duck'

n
19809


In [100]:
%%sql
DROP TABLE IF EXISTS processo_senado;

CREATE TABLE processo_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year
    FROM bronze_senado_processo
),
flat AS (
    SELECT
        -- identificadores principais
        CAST(jget1(j, '$.id') AS BIGINT)                       AS id_processo,
        CAST(jget1(j, '$.codigoMateria') AS BIGINT)            AS codigo_materia,
        CAST(jget1(j, '$.idProcessoCasaInicial') AS BIGINT)    AS id_processo_casa_inicial,

        jget1(j, '$.identificacao')                            AS identificacao,
        jget1(j, '$.identificacaoProcessoInicial')             AS identificacao_processo_inicial,
        jget1(j, '$.identificacaoExterna')                     AS identificacao_externa,

        -- campos simples de nível superior
        CAST(jget1(j, '$.ano') AS INTEGER)                     AS ano,
        jget1(j, '$.casaIdentificadora')                       AS casa_identificadora,
        jget1(j, '$.siglaCasaIniciadora')                      AS sigla_casa_iniciadora,
        jget1(j, '$.siglaEnteIdentificador')                   AS sigla_ente_identificador,
        jget1(j, '$.descricaoSigla')                           AS descricao_sigla,
        jget1(j, '$.sigla')                                    AS sigla,
        jget1(j, '$.numero')                                   AS numero,
        jget1(j, '$.objetivo')                                 AS objetivo,
        jget1(j, '$.tramitando')                               AS tramitando,

        -- conteúdo principal
        CAST(jget1(j, '$.conteudo.id') AS BIGINT)              AS id_conteudo,
        CAST(jget1(j, '$.conteudo.idTipo') AS BIGINT)          AS id_tipo_conteudo,
        jget1(j, '$.conteudo.siglaTipo')                       AS sigla_tipo_conteudo,
        jget1(j, '$.conteudo.tipo')                            AS tipo_conteudo,
        jget1(j, '$.conteudo.tipoNormaIndicada')               AS tipo_norma_indicada,
        jget1(j, '$.conteudo.ementa')                          AS ementa,
        jget1(j, '$.conteudo.explicacaoEmenta')                AS explicacao_ementa,

        -- deliberação final
        CAST(jget1(j, '$.deliberacao.idDestino') AS BIGINT)    AS deliberacao_id_destino,
        jget1(j, '$.deliberacao.siglaDestino')                 AS deliberacao_sigla_destino,
        jget1(j, '$.deliberacao.tipoDeliberacao')              AS deliberacao_tipo,
        jget1(j, '$.deliberacao.siglaTipo')                    AS deliberacao_sigla_tipo,
        CAST(jget1(j, '$.deliberacao.data') AS DATE)           AS deliberacao_data,
        jget1(j, '$.deliberacao.destino')                      AS deliberacao_destino,

        -- documento associado
        CAST(jget1(j, '$.documento.id') AS BIGINT)             AS id_documento,
        jget1(j, '$.documento.siglaTipo')                      AS documento_sigla_tipo,
        jget1(j, '$.documento.tipo')                           AS documento_tipo,
        jget1(j, '$.documento.indexacao')                      AS documento_indexacao,
        jget1(j, '$.documento.resumoAutoria')                  AS documento_resumo_autoria,
        CAST(jget1(j, '$.documento.dataApresentacao') AS DATE) AS documento_data_apresentacao,
        CAST(jget1(j, '$.documento.dataLeitura') AS DATE)      AS documento_data_leitura,

        -- norma gerada
        CAST(jget1(j, '$.normaGerada.codigo') AS BIGINT)       AS norma_codigo,
        jget1(j, '$.normaGerada.numero')                       AS norma_numero,
        jget1(j, '$.normaGerada.siglaTipo')                    AS norma_sigla_tipo,
        jget1(j, '$.normaGerada.tipo')                         AS norma_tipo,
        jget1(j, '$.normaGerada.descricao')                    AS norma_descricao,
        jget1(j, '$.normaGerada.siglaVeiculoPublicacao')       AS norma_sigla_veiculo,
        jget1(j, '$.normaGerada.veiculoPublicacao')            AS norma_veiculo,
        CAST(jget1(j, '$.normaGerada.numero') AS INTEGER)      AS norma_numero_int,
        CAST(jget1(j, '$.normaGerada.anoAssinatura') AS INTEGER) AS norma_ano_assinatura,
        CAST(jget1(j, '$.normaGerada.dataAssinatura') AS DATE) AS norma_data_assinatura,
        CAST(jget1(j, '$.normaGerada.dataPublicacao') AS DATE) AS norma_data_publicacao,

        year AS year_snapshot
    FROM base
)
SELECT *
FROM flat
WHERE id_processo IS NOT NULL;


DROP TABLE IF EXISTS tipo_deliberacao_senado;

CREATE TABLE tipo_deliberacao_senado AS
WITH base AS (
  SELECT
    deliberacao_sigla_tipo   AS sigla_tipo_deliberacao,
    deliberacao_tipo         AS descricao_tipo_deliberacao,
    -- campos auxiliares úteis (opc.)
    deliberacao_id_destino   AS id_destino,
    deliberacao_sigla_destino AS sigla_destino,
    deliberacao_destino      AS destino,
    year_snapshot
  FROM processo_senado
  WHERE deliberacao_sigla_tipo IS NOT NULL
)
SELECT
  sigla_tipo_deliberacao,
  arg_max(descricao_tipo_deliberacao, year_snapshot) AS descricao_tipo_deliberacao,
  arg_max(id_destino,               year_snapshot)   AS id_destino,
  arg_max(sigla_destino,            year_snapshot)   AS sigla_destino,
  arg_max(destino,                  year_snapshot)   AS destino,
  max(year_snapshot)                                AS year_snapshot
FROM base
GROUP BY sigla_tipo_deliberacao;

DROP TABLE IF EXISTS tipo_conteudo_senado;

CREATE TABLE tipo_conteudo_senado AS
WITH base AS (
  SELECT
    id_tipo_conteudo       AS id_tipo_conteudo,      -- chave numérica
    sigla_tipo_conteudo    AS sigla_tipo_conteudo,
    tipo_conteudo          AS descricao_tipo_conteudo,
    tipo_norma_indicada    AS tipo_norma_indicada,   -- pode ser NULL dependendo do caso
    year_snapshot
  FROM processo_senado
  WHERE id_tipo_conteudo IS NOT NULL
)
SELECT
  id_tipo_conteudo,
  arg_max(sigla_tipo_conteudo,    year_snapshot) AS sigla_tipo_conteudo,
  arg_max(descricao_tipo_conteudo,year_snapshot) AS descricao_tipo_conteudo,
  arg_max(tipo_norma_indicada,    year_snapshot) AS tipo_norma_indicada,
  max(year_snapshot)                             AS year_snapshot
FROM base
GROUP BY id_tipo_conteudo;

DROP VIEW IF EXISTS bronze_senado_processo;


Running query in 'duck'

Success


## 2.2 Modelos factuais

### 2.2.1 Emendas

In [101]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_emendas AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/senado/emendas/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'emendas';

SELECT COUNT(*) AS n FROM bronze_senado_emendas;

Running query in 'duck'

n
6463


In [102]:
%%sql

DROP TABLE IF EXISTS emendas_senado;

CREATE TABLE emendas_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year
    FROM bronze_senado_emendas
),
flat AS (
    SELECT
        -- chaves e metadados
        CAST(jget1(j, '$.id')                AS BIGINT)  AS id_emenda,
        CAST(jget1(j, '$.idCiEmenda')        AS BIGINT)  AS id_ci_emenda,
        CAST(jget1(j, '$.idCiEmendado')      AS BIGINT)  AS id_ci_emendado,
        CAST(jget1(j, '$.idDocumentoEmenda') AS BIGINT)  AS id_documento_emenda,
        CAST(jget1(j, '$.idProcesso')        AS BIGINT)  AS id_processo,

        -- identificação/descrição
        jget1(j, '$.identificacao')                      AS identificacao,
        CAST(jget1(j, '$.numero') AS INTEGER)            AS numero,
        jget1(j, '$.autoria')                             AS autoria,
        jget1(j, '$.descricaoDocumentoEmenda')            AS descricao_documento_emenda,
        jget1(j, '$.tipo')                                AS tipo_emenda,
        jget1(j, '$.turnoApresentacao')                   AS turno_apresentacao,

        -- casa/colegiado
        jget1(j, '$.casa')                                AS casa,
        CAST(jget1(j, '$.codigoColegiado') AS BIGINT)     AS codigo_colegiado,
        jget1(j, '$.siglaColegiado')                      AS sigla_colegiado,
        jget1(j, '$.nomeColegiado')                       AS nome_colegiado,

        -- documentos e datas
        CAST(jget1(j, '$.dataApresentacao') AS DATE)      AS data_apresentacao,
        jget1(j, '$.urlDocumentoEmenda')                  AS url_documento_emenda,

        -- snapshot
        year AS year_snapshot
    FROM base
)
SELECT *
FROM flat
WHERE id_emenda IS NOT NULL;


Running query in 'duck'

Count
6463


In [103]:
%%sql

DROP TABLE IF EXISTS tipo_emendas_senado;

CREATE TABLE tipo_emendas_senado AS
WITH base AS (
    SELECT
        tipo_emenda,
        year_snapshot
    FROM emendas_senado
    WHERE tipo_emenda IS NOT NULL
)
SELECT
    tipo_emenda,
    max(year_snapshot) AS year_snapshot
FROM base
GROUP BY tipo_emenda;


Running query in 'duck'

Count
2


In [104]:
%%sql
DROP VIEW IF EXISTS bronze_senado_emendas;

Running query in 'duck'

Success


### 2.2.2 Relatorias

In [105]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_relatorias AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/senado/relatorias/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'relatorias';

SELECT COUNT(*) AS n FROM bronze_senado_relatorias;

Running query in 'duck'

n
2029


In [106]:
%%sql
DROP TABLE IF EXISTS relatorias_senado;

CREATE TABLE relatorias_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year
    FROM bronze_senado_relatorias
)
SELECT
    -- chaves e relacionamentos
    CAST(jget1(j, '$.id')                AS BIGINT)   AS id_relatoria,
    CAST(jget1(j, '$.idProcesso')        AS BIGINT)   AS id_processo,
    CAST(jget1(j, '$.codigoMateria')     AS BIGINT)   AS codigo_materia,
    CAST(jget1(j, '$.codigoParlamentar') AS BIGINT)   AS codigo_parlamentar,

    -- colegiado
    CAST(jget1(j, '$.codigoColegiado')      AS BIGINT)  AS codigo_colegiado,
    CAST(jget1(j, '$.codigoTipoColegiado')  AS BIGINT)  AS codigo_tipo_colegiado,
    jget1(j, '$.siglaColegiado')                         AS sigla_colegiado,
    jget1(j, '$.nomeColegiado')                          AS nome_colegiado,

    -- processo
    jget1(j, '$.autoriaProcesso')                        AS autoria_processo,
    jget1(j, '$.identificacaoProcesso')                  AS identificacao_processo,
    jget1(j, '$.ementaProcesso')                         AS ementa_processo,
    CAST(jget1(j, '$.numeroAutuacao') AS INTEGER)        AS numero_autuacao,
    jget1(j, '$.tramitando')                              AS tramitando,

    -- casa/relator
    jget1(j, '$.siglaCasa')                               AS sigla_casa,
    jget1(j, '$.casaRelator')                             AS casa_relator,
    jget1(j, '$.descricaoTipoRelator')                    AS descricao_tipo_relator,
    CAST(jget1(j, '$.idTipoRelator') AS INTEGER)          AS id_tipo_relator,
    jget1(j, '$.descricaoTipoEncerramento')               AS descricao_tipo_encerramento,

    -- parlamentar
    jget1(j, '$.formaTratamentoParlamentar')              AS forma_tratamento_parlamentar,
    jget1(j, '$.nomeParlamentar')                         AS nome_parlamentar,
    jget1(j, '$.nomeCompleto')                            AS nome_completo,
    jget1(j, '$.siglaPartidoParlamentar')                 AS sigla_partido_parlamentar,
    jget1(j, '$.ufParlamentar')                           AS uf_parlamentar,
    jget1(j, '$.sexoParlamentar')                         AS sexo_parlamentar,
    jget1(j, '$.emailParlamentar')                        AS email_parlamentar,
    jget1(j, '$.urlFotoParlamentar')                      AS url_foto_parlamentar,
    jget1(j, '$.urlPaginaParlamentar')                    AS url_pagina_parlamentar,

    -- datas
    CAST(jget1(j, '$.dataApresentacaoProcesso') AS DATETIME) AS data_apresentacao_processo,
    CAST(jget1(j, '$.dataDesignacao')           AS DATETIME) AS data_designacao,
    CAST(jget1(j, '$.dataDestituicao')          AS DATETIME) AS data_destituicao,
    CAST(jget1(j, '$.dataFimColegiado')         AS DATETIME) AS data_fim_colegiado,

    -- snapshot
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_senado_relatorias;

Running query in 'duck'

Success


### 2.2.3 Votações e votos

In [107]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_votacoes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/senado/votacoes/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'votacoes';

SELECT COUNT(*) AS n FROM bronze_senado_votacoes;

Running query in 'duck'

n
604


In [108]:
%%sql
DROP TABLE IF EXISTS votacoes_senado;

CREATE TABLE votacoes_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year AS year_snapshot
    FROM bronze_senado_votacoes
)
SELECT
    CAST(jget1(j, '$.codigoSessaoVotacao') AS BIGINT) AS id_votacao,
    CAST(jget1(j, '$.codigoMateria') AS BIGINT) AS id_materia,
    CAST(jget1(j, '$.idProcesso') AS BIGINT) AS id_processo,
    jget1(j, '$.identificacao') AS identificacao,

    jget1(j, '$.sigla') AS sigla,
    jget1(j, '$.numero') AS numero,
    CAST(jget1(j, '$.ano') AS INTEGER) AS ano,

    CAST(jget1(j, '$.codigoSessao') AS BIGINT) AS codigo_sessao,
    CAST(jget1(j, '$.numeroSessao') AS INTEGER) AS numero_sessao,
    CAST(jget1(j, '$.sequencialSessao') AS INTEGER) AS sequencial_sessao,
    jget1(j, '$.siglaTipoSessao') AS sigla_tipo_sessao,
    jget1(j, '$.casaSessao') AS casa_sessao,
    CAST(jget1(j, '$.codigoSessaoLegislativa') AS BIGINT) AS codigo_sessao_legislativa,

    CAST(jget1(j, '$.dataApresentacao') AS TIMESTAMP) AS data_apresentacao,
    CAST(jget1(j, '$.dataSessao') AS TIMESTAMP) AS data_sessao,

    jget1(j, '$.descricaoVotacao') AS descricao_votacao,
    jget1(j, '$.ementa') AS ementa,

    jget1(j, '$.resultadoVotacao') AS resultado_votacao,
    CAST(jget1(j, '$.totalVotosSim') AS INTEGER) AS total_votos_sim,
    CAST(jget1(j, '$.totalVotosNao') AS INTEGER) AS total_votos_nao,
    CAST(jget1(j, '$.totalVotosAbstencao') AS INTEGER) AS total_votos_abstencao,
    jget1(j, '$.votacaoSecreta') AS votacao_secreta,

    -- informe legislativo expandido
    CAST(jget1(j, '$.informeLegislativo.id') AS BIGINT) AS id_informe,
    CAST(jget1(j, '$.informeLegislativo.idEvento') AS BIGINT) AS id_evento,
    CAST(jget1(j, '$.informeLegislativo.codigoColegiado') AS BIGINT) AS codigo_colegiado,
    jget1(j, '$.informeLegislativo.nomeColegiado') AS nome_colegiado,
    jget1(j, '$.informeLegislativo.siglaColegiado') AS sigla_colegiado,
    CAST(jget1(j, '$.informeLegislativo.data') AS TIMESTAMP) AS data_informe,
    jget1(j, '$.informeLegislativo.texto') AS texto_informe,

    year_snapshot
FROM base;


Running query in 'duck'

Count
604


In [109]:
%%sql
DROP TABLE IF EXISTS votos_senado;

CREATE TABLE votos_senado AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot
  FROM bronze_senado_votacoes
),
exploded AS (
  SELECT
    CAST(jget1(b.j, '$.codigoVotacaoSve') AS BIGINT)        AS codigo_votacao_sve,
    CAST(jget1(b.j, '$.codigoSessaoVotacao') AS BIGINT)     AS codigo_sessao_votacao,
    CAST(jget1(b.j, '$.codigoMateria') AS BIGINT)           AS codigo_materia,
    jget1(b.j, '$.identificacao')                           AS identificacao_materia,

    -- Campos do voto
    CAST(json_extract_string(e.value, '$.codigoParlamentar') AS BIGINT) AS codigo_parlamentar,
    json_extract_string(e.value, '$.nomeParlamentar')        AS nome_parlamentar,
    json_extract_string(e.value, '$.sexoParlamentar')        AS sexo_parlamentar,
    json_extract_string(e.value, '$.siglaPartidoParlamentar')AS sigla_partido_parlamentar,
    json_extract_string(e.value, '$.siglaUFParlamentar')     AS sigla_uf_parlamentar,
    json_extract_string(e.value, '$.siglaVotoParlamentar')   AS sigla_voto_parlamentar,
    json_extract_string(e.value, '$.descricaoVotoParlamentar') AS descricao_voto_parlamentar,

    -- Snapshot
    b.year_snapshot
  FROM base b
  -- explode apenas quando existir a lista
  CROSS JOIN json_each(b.j, '$.votos') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_voto,
        codigo_votacao_sve,
        codigo_sessao_votacao,
        codigo_materia,
        identificacao_materia,
        codigo_parlamentar,
        nome_parlamentar,
        sexo_parlamentar,
        sigla_partido_parlamentar,
        sigla_uf_parlamentar,
        sigla_voto_parlamentar,
        descricao_voto_parlamentar,
        year_snapshot
    FROM exploded
)
SELECT * FROM numbered;


Running query in 'duck'

Count
48877


In [110]:
%%sql

DROP VIEW IF EXISTS bronze_senado_votacoes;

Running query in 'duck'

Success


### 2.2.4 Votações / Colegiado

## 2.3 Outras relações entre tabelas

### 2.3.1 Processos: autoria iniciativa, autoria documento

In [111]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_processo AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/senado/processo/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'processo';

SELECT COUNT(*) AS n FROM bronze_senado_processo;

Running query in 'duck'

n
19809


In [112]:
%%sql
DROP TABLE IF EXISTS autoria_iniciativa_senado;

CREATE TABLE autoria_iniciativa_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(jget1(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(e.value, '$.idEnte') AS BIGINT) AS id_ente,
        json_extract_string(e.value, '$.autor') AS autor,
        json_extract_string(e.value, '$.casaEnte') AS casa_ente,
        CAST(json_extract_string(e.value, '$.codigoParlamentar') AS BIGINT) AS codigo_parlamentar,
        json_extract_string(e.value, '$.descricaoTipo') AS descricao_tipo,
        json_extract_string(e.value, '$.ente') AS ente,
        CAST(json_extract_string(e.value, '$.ordem') AS INTEGER) AS ordem,
        json_extract_string(e.value, '$.outrosAutoresNaoInformados') AS outros_autores_nao_informados,
        json_extract_string(e.value, '$.siglaEnte') AS sigla_ente,
        json_extract_string(e.value, '$.siglaTipo') AS sigla_tipo,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.autoriaIniciativa') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_autoria_iniciativa,
        id_processo,
        codigo_parlamentar,
        descricao_tipo,
        ente,
        ordem,
        outros_autores_nao_informados,
        sigla_ente,
        sigla_tipo,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
35736


In [113]:
%%sql
DROP TABLE IF EXISTS autoria_documento_senado;

CREATE TABLE autoria_documento_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) as j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) as id_processo,
        CAST(json_extract_string(payload_json, '$.documento.id') AS BIGINT) as id_documento
    FROM bronze_senado_processo
),
exploded as (
    SELECT 
    b.id_processo,
    b.id_documento,
    json_extract_string(e, '$.autor') as autor,
    json_extract_string(e, '$.siglaTipo') as sigla_tipo,
    json_extract_string(e, '$.descricaoTipo') as descricao_tipo,
    CAST(json_extract_string(e, '$.ordem') AS INTEGER) as ordem,
    json_extract_string(e, '$.outrosAutoresNaoInformados') as outros_autores_nao_informados,
    CAST(json_extract_string(e, '$.idEnte') AS BIGINT) as id_ente,
    json_extract_string(e, '$.siglaEnte') as sigla_ente,
    json_extract_string(e, '$.casaEnte') as casa_ente,
    json_extract_string(e, '$.ente') as ente,
    json_extract_string(e, '$.cargo') as cargo,
    json_extract_string(e, '$.siglaCargo') as siglaCargo,
    json_extract_string(e, '$.partido') as partido,
    json_extract_string(e, '$.sexo') as sexo,
    json_extract_string(e, '$.uf') as uf,
    b.year_snapshot,
    FROM base b
    CROSS JOIN json_each(b.j, '$.documento.autoria') as e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_autoria_documento,
        id_processo,
        id_documento,
        autor,
        sigla_tipo,
        descricao_tipo,
        ordem,
        outros_autores_nao_informados,
        id_ente,
        sigla_ente,
        casa_ente,
        ente,
        cargo,
        siglaCargo,
        partido,
        sexo,
        uf,
        year_snapshot
    FROM exploded
)
SELECT * FROM numbered;

Running query in 'duck'

Count
36479


In [114]:
%%sql
DROP TABLE IF EXISTS situacoes_senado;

CREATE TABLE situacoes_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
autuacoes AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(a.value, '$.numero') AS INTEGER) AS numero_autuacao,
        a.value AS autuacao_json,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.autuacoes') AS a
),
situacoes AS (
    SELECT
        a.id_processo,
        a.numero_autuacao,
        CAST(json_extract_string(s.value, '$.idTipo') AS INTEGER) AS id_tipo_situacao,
        json_extract_string(s.value, '$.sigla') AS sigla_situacao,
        json_extract_string(s.value, '$.descricao') AS descricao_situacao,
        TRY_CAST(json_extract_string(s.value, '$.inicio') AS DATE) AS data_inicio,
        TRY_CAST(json_extract_string(s.value, '$.fim') AS DATE) AS data_fim,
        a.year_snapshot
    FROM autuacoes a
    CROSS JOIN json_each(a.autuacao_json, '$.situacoes') AS s
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_situacao,
        id_processo,
        numero_autuacao,
        id_tipo_situacao,
        sigla_situacao,
        descricao_situacao,
        data_inicio,
        data_fim,
        year_snapshot
    FROM situacoes
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
93321


In [115]:
%%sql
DROP TABLE IF EXISTS despachos_senado;

CREATE TABLE despachos_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(d.value, '$.id') AS BIGINT) AS id_despacho,
        TRY_CAST(json_extract_string(d.value, '$.data') AS DATE) AS data_despacho,
        json_extract_string(d.value, '$.cancelado') AS cancelado,
        json_extract_string(d.value, '$.tipoMotivacao') AS tipo_motivacao,
        json_extract_string(d.value, '$.siglaTipoMotivacao') AS sigla_tipo_motivacao,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.despachos') AS d
)
SELECT
    id_processo,
    id_despacho,
    data_despacho,
    cancelado,
    tipo_motivacao,
    sigla_tipo_motivacao,
    year_snapshot
FROM exploded;


Running query in 'duck'

Count
7428


In [116]:
%%sql
DROP TABLE IF EXISTS outros_numeros_senado;

CREATE TABLE outros_numeros_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(o.value, '$.idOutroProcesso') AS BIGINT) AS id_outro_processo,
        CAST(json_extract_string(o.value, '$.ano') AS INTEGER) AS ano,
        json_extract_string(o.value, '$.casaIdentificadora') AS casa_identificadora,
        json_extract_string(o.value, '$.enteIdentificador') AS ente_identificador,
        json_extract_string(o.value, '$.sigla') AS sigla,
        json_extract_string(o.value, '$.numero') AS numero,
        json_extract_string(o.value, '$.siglaEnteIdentificador') AS sigla_ente_identificador,
        json_extract_string(o.value, '$.externaAoCongresso') AS externa_ao_congresso,
        json_extract_string(o.value, '$.tramitando') AS tramitando,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.outrosNumeros') AS o
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_outro_numero,
        id_processo,
        id_outro_processo,
        ano,
        casa_identificadora,
        ente_identificador,
        sigla,
        numero,
        sigla_ente_identificador,
        externa_ao_congresso,
        tramitando,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;

Running query in 'duck'

Count
4062


In [117]:
%%sql
DROP TABLE IF EXISTS processos_relacionados_senado;

CREATE TABLE processos_relacionados_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(p.value, '$.idOutroProcesso') AS BIGINT) AS id_outro_processo,
        CAST(json_extract_string(p.value, '$.ano') AS INTEGER) AS ano,
        json_extract_string(p.value, '$.casaIdentificadora') AS casa_identificadora,
        json_extract_string(p.value, '$.enteIdentificador') AS ente_identificador,
        json_extract_string(p.value, '$.sigla') AS sigla,
        json_extract_string(p.value, '$.numero') AS numero,
        json_extract_string(p.value, '$.siglaEnteIdentificador') AS sigla_ente_identificador,
        json_extract_string(p.value, '$.tipoRelacao') AS tipo_relacao,
        json_extract_string(p.value, '$.tramitando') AS tramitando,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.processosRelacionados') AS p
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_processo_relacionado,
        id_processo,
        id_outro_processo,
        ano,
        casa_identificadora,
        ente_identificador,
        sigla,
        numero,
        sigla_ente_identificador,
        tipo_relacao,
        tramitando,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;

Running query in 'duck'

Count
8363


In [118]:
%%sql
DROP TABLE IF EXISTS documento_autoria_senado;

CREATE TABLE documento_autoria_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(a.value, '$.idEnte') AS BIGINT) AS id_ente,
        json_extract_string(a.value, '$.autor') AS autor,
        json_extract_string(a.value, '$.casaEnte') AS casa_ente,
        CAST(json_extract_string(a.value, '$.codigoParlamentar') AS BIGINT) as codigo_parlamentar,
        json_extract_string(a.value, '$.descricaoTipo') AS descricao_tipo,
        json_extract_string(a.value, '$.ente') AS ente,
        CAST(json_extract_string(a.value, '$.ordem') AS INTEGER) AS ordem,
        json_extract_string(a.value, '$.outrosAutoresNaoInformados') AS outros_autores_nao_informados,
        json_extract_string(a.value, '$.siglaEnte') AS sigla_ente,
        json_extract_string(a.value, '$.siglaTipo') AS sigla_tipo,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.documento.autoria') AS a
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_documento_autoria,
        id_processo,
        id_ente,
        autor,
        codigo_parlamentar,
        descricao_tipo,
        ente,
        ordem,
        outros_autores_nao_informados,
        sigla_ente,
        sigla_tipo,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
36479


In [119]:
%%sql
DROP TABLE IF EXISTS autuacoes_senado;

CREATE TABLE autuacoes_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON)                                   AS j,
        year                                                             AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT)        AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        -- array position within $.autuacoes becomes a stable, deterministic PK within the processo
        CAST(a.key AS INTEGER) + 1                                       AS autuacao_idx,   -- 1-based
        json_extract_string(a.value, '$.descricao')                      AS descricao_autuacao,
        CAST(json_extract_string(a.value, '$.idEnteControleAtual') AS BIGINT)
                                                                          AS id_ente_controle_atual,
        json_extract_string(a.value, '$.nomeEnteControleAtual')          AS nome_ente_controle_atual,
        json_extract_string(a.value, '$.siglaEnteControleAtual')         AS sigla_ente_controle_atual,
        CAST(json_extract_string(a.value, '$.numero') AS INTEGER)        AS numero_autuacao,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.autuacoes') AS a
)
SELECT
    id_processo,
    autuacao_idx,
    descricao_autuacao,
    id_ente_controle_atual,
    nome_ente_controle_atual,
    sigla_ente_controle_atual,
    numero_autuacao,
    year_snapshot
FROM exploded;


Running query in 'duck'

Count
19816


In [120]:
%%sql
DROP TABLE IF EXISTS informes_legislativos_senado;

CREATE TABLE informes_legislativos_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON)                            AS j,
        year                                                      AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
autuacoes AS (
    SELECT
        b.id_processo,
        b.year_snapshot,
        a.value AS autuacao_json
    FROM base b
    CROSS JOIN json_each(b.j, '$.autuacoes') AS a
),
informes AS (
    SELECT
        a.id_processo,
        a.year_snapshot,
        i.value AS informe_json
    FROM autuacoes a
    CROSS JOIN json_each(a.autuacao_json, '$.informesLegislativos') AS i
)
SELECT
    ROW_NUMBER() OVER ()                                         AS id_informe_legislativo, -- auto increment PK
    id_processo,

    CAST(json_extract_string(informe_json, '$.id') AS BIGINT)    AS id_informe,
    TRY_CAST(json_extract_string(informe_json, '$.data') AS DATE) AS data_informe,
    json_extract_string(informe_json, '$.descricao')             AS descricao,

    -- situação iniciada (optional)
    CAST(json_extract_string(informe_json, '$.idSituacaoIniciada') AS BIGINT) AS id_situacao_iniciada,
    json_extract_string(informe_json, '$.siglaSituacaoIniciada')  AS sigla_situacao_iniciada,

    -- enteAdministrativo
    json_extract_string(informe_json, '$.enteAdministrativo.casa')               AS ente_adm_casa,
    CAST(json_extract_string(informe_json, '$.enteAdministrativo.id') AS BIGINT) AS ente_adm_id,
    json_extract_string(informe_json, '$.enteAdministrativo.nome')               AS ente_adm_nome,
    json_extract_string(informe_json, '$.enteAdministrativo.sigla')              AS ente_adm_sigla,

    -- colegiado
    json_extract_string(informe_json, '$.colegiado.casa')          AS colegiado_casa,
    CAST(json_extract_string(informe_json, '$.colegiado.codigo') AS BIGINT) AS colegiado_codigo,
    json_extract_string(informe_json, '$.colegiado.nome')          AS colegiado_nome,
    json_extract_string(informe_json, '$.colegiado.sigla')         AS colegiado_sigla,

    year_snapshot
FROM informes;


Running query in 'duck'

Count
318424


In [121]:
%%sql
DROP TABLE IF EXISTS informes_documentos_associados_senado;

CREATE TABLE informes_documentos_associados_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
autuacoes AS (
    SELECT
        b.id_processo,
        CAST(a.key AS INTEGER)                    AS autuacao_ordem,
        a.value                                   AS autuacao_json,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.autuacoes') AS a
),
informes AS (
    SELECT
        a.id_processo,
        a.autuacao_ordem,
        CAST(i.key AS INTEGER)                    AS informe_ordem,
        CAST(json_extract_string(i.value, '$.id') AS BIGINT) AS id_informe,
        i.value                                   AS informe_json,
        a.year_snapshot
    FROM autuacoes a
    CROSS JOIN json_each(a.autuacao_json, '$.informesLegislativos') AS i
),
docs AS (
    SELECT
        inf.id_processo,
        inf.id_informe,
        inf.autuacao_ordem,
        inf.informe_ordem,
        CAST(d.key AS INTEGER)                    AS documento_ordem,
        CAST(json_extract_string(d.value, '$.id') AS BIGINT)   AS id_documento,
        json_extract_string(d.value, '$.siglaTipo')            AS sigla_tipo_documento,
        json_extract_string(d.value, '$.tipo')                 AS tipo_documento,
        json_extract_string(d.value, '$.identificacao')        AS identificacao,
        TRY_CAST(json_extract_string(d.value, '$.data') AS TIMESTAMP) AS data_documento,
        json_extract_string(d.value, '$.autoria')              AS autoria_documento,
        json_extract_string(d.value, '$.url')                  AS url_documento,
        inf.year_snapshot
    FROM informes inf
    CROSS JOIN json_each(inf.informe_json, '$.documentosAssociados') AS d
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_documento_associado,
        id_processo,
        id_informe,
        id_documento,
        autuacao_ordem,
        informe_ordem,
        documento_ordem,
        sigla_tipo_documento,
        tipo_documento,
        identificacao,
        data_documento,
        autoria_documento,
        url_documento,
        year_snapshot
    FROM docs
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
92610


In [122]:
%%sql
-- Movimentações dentro de cada autuação do processo
DROP TABLE IF EXISTS movimentacoes_senado;

CREATE TABLE movimentacoes_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON)                                           AS j,
        year                                                                     AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT)                AS id_processo
    FROM bronze_senado_processo
),
autuacoes AS (  -- explode top-level autuações and keep their array index as autuacao_idx
    SELECT
        b.id_processo,
        CAST(a.key AS INTEGER)                                                   AS autuacao_idx,
        a.value                                                                  AS autuacao_json,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.autuacoes') AS a
),
exploded AS (  -- explode movimentações under each autuação
    SELECT
        a.id_processo,
        a.autuacao_idx,
        CAST(m.key AS INTEGER)                                                   AS movimentacao_idx,
        CAST(json_extract_string(m.value, '$.id') AS BIGINT)                     AS id_movimentacao,
        TRY_CAST(json_extract_string(m.value, '$.dataEnvio') AS TIMESTAMP)       AS data_envio,
        TRY_CAST(json_extract_string(m.value, '$.dataRecebimento') AS TIMESTAMP) AS data_recebimento,

        -- Ente de origem
        json_extract_string(m.value, '$.enteOrigem.casa')                        AS ente_origem_casa,
        CAST(json_extract_string(m.value, '$.enteOrigem.id') AS BIGINT)          AS ente_origem_id,
        json_extract_string(m.value, '$.enteOrigem.nome')                        AS ente_origem_nome,
        json_extract_string(m.value, '$.enteOrigem.sigla')                       AS ente_origem_sigla,

        -- Ente de destino
        json_extract_string(m.value, '$.enteDestino.casa')                       AS ente_destino_casa,
        CAST(json_extract_string(m.value, '$.enteDestino.id') AS BIGINT)         AS ente_destino_id,
        json_extract_string(m.value, '$.enteDestino.nome')                       AS ente_destino_nome,
        json_extract_string(m.value, '$.enteDestino.sigla')                      AS ente_destino_sigla,

        -- Colegiado de destino (pode vir vazio {})
        json_extract_string(m.value, '$.colegiadoDestino.casa')                  AS colegiado_destino_casa,
        CAST(json_extract_string(m.value, '$.colegiadoDestino.codigo') AS BIGINT) AS colegiado_destino_codigo,
        json_extract_string(m.value, '$.colegiadoDestino.nome')                  AS colegiado_destino_nome,
        json_extract_string(m.value, '$.colegiadoDestino.sigla')                 AS colegiado_destino_sigla,

        a.year_snapshot
    FROM autuacoes a
    CROSS JOIN json_each(a.autuacao_json, '$.movimentacoes') AS m
)
SELECT
    id_processo,
    autuacao_idx,            -- FK para autuacoes_senado (id_processo, autuacao_idx)
    movimentacao_idx,        -- posição dentro da autuação (opcional como parte da PK composta)
    id_movimentacao,
    data_envio,
    data_recebimento,
    ente_origem_casa,
    ente_origem_id,
    ente_origem_nome,
    ente_origem_sigla,
    ente_destino_casa,
    ente_destino_id,
    ente_destino_nome,
    ente_destino_sigla,
    colegiado_destino_casa,
    colegiado_destino_codigo,
    colegiado_destino_nome,
    colegiado_destino_sigla,
    year_snapshot
FROM exploded;


Running query in 'duck'

Count
93707


In [123]:
%%sql
DROP TABLE IF EXISTS providencias_senado;

CREATE TABLE providencias_senado AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
  FROM bronze_senado_processo
),
despachos AS (
  SELECT
    b.id_processo,
    CAST(json_extract_string(d.value, '$.id') AS BIGINT) AS id_despacho,
    d.value AS despacho_json,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.despachos') AS d
),
exploded AS (
  SELECT
    d.id_processo,
    d.id_despacho,
    CAST(json_extract_string(p.value, '$.id') AS BIGINT)            AS id_providencia,
    json_extract_string(p.value, '$.descricao')                      AS descricao,
    json_extract_string(p.value, '$.tipo')                           AS tipo,
    json_extract_string(p.value, '$.analiseConteudo')                AS analise_conteudo,
    json_extract_string(p.value, '$.analiseTempo')                   AS analise_tempo,
    CAST(json_extract_string(p.value, '$.ordem') AS INTEGER)         AS ordem,
    json_extract_string(p.value, '$.reexame')                        AS reexame,       -- "Sim"/"Não"
    d.year_snapshot
  FROM despachos d
  CROSS JOIN json_each(d.despacho_json, '$.providencias') AS p
)
SELECT
  id_processo,
  id_despacho,
  id_providencia,
  descricao,
  tipo,
  analise_conteudo,
  analise_tempo,
  ordem,
  reexame,
  year_snapshot
FROM exploded;


Running query in 'duck'

Count
7744


In [124]:
%%sql
DROP TABLE IF EXISTS unidades_destinatarias_senado;

CREATE TABLE unidades_destinatarias_senado AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
  FROM bronze_senado_processo
),
desp AS (
  SELECT
    b.id_processo,
    b.year_snapshot,
    CAST(json_extract_string(d.value, '$.id') AS BIGINT) AS id_despacho,
    d.value AS despacho_json
  FROM base b
  CROSS JOIN json_each(b.j, '$.despachos') AS d
),
prov AS (
  SELECT
    d.id_processo,
    d.year_snapshot,
    d.id_despacho,
    CAST(json_extract_string(p.value, '$.id') AS BIGINT) AS id_providencia,
    p.value AS providencia_json
  FROM desp d
  CROSS JOIN json_each(d.despacho_json, '$.providencias') AS p
),
unid AS (
  SELECT
    p.id_processo,
    p.year_snapshot,
    p.id_despacho,
    p.id_providencia,
    CAST(json_extract_string(u.value, '$.colegiado.codigo') AS BIGINT) AS colegiado_codigo,
    json_extract_string(u.value, '$.colegiado.casa')  AS colegiado_casa,
    json_extract_string(u.value, '$.colegiado.nome')  AS colegiado_nome,
    json_extract_string(u.value, '$.colegiado.sigla') AS colegiado_sigla,
    CAST(json_extract_string(u.value, '$.ordem') AS INTEGER) AS ordem,
    json_extract_string(u.value, '$.tipoAnaliseDeliberacao') AS tipo_analise_deliberacao
  FROM prov p
  CROSS JOIN json_each(p.providencia_json, '$.unidadesDestinatarias') AS u
),
numbered AS (
  SELECT
    ROW_NUMBER() OVER () AS id_unidade_destinataria,
    id_processo,
    id_despacho,
    id_providencia,
    colegiado_casa,
    colegiado_codigo,
    colegiado_nome,
    colegiado_sigla,
    ordem,
    tipo_analise_deliberacao,
    year_snapshot
  FROM unid
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
9667


In [125]:
%%sql
DROP TABLE IF EXISTS encontro_legislativo_senado;

CREATE TABLE encontro_legislativo_senado AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
  FROM bronze_senado_processo
),
desp AS (
  SELECT
    b.id_processo,
    CAST(json_extract_string(d.value, '$.id') AS BIGINT) AS id_despacho,
    d.value AS despacho_json,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.despachos') AS d
),
encontro AS (
  SELECT
    id_processo,
    id_despacho,
    CAST(json_extract_string(despacho_json, '$.encontroLegislativo.id') AS BIGINT)          AS id_encontro_legislativo,
    TRY_CAST(json_extract_string(despacho_json, '$.encontroLegislativo.data') AS DATE)      AS data_encontro,
    json_extract_string(despacho_json, '$.encontroLegislativo.tipo')                         AS tipo_encontro,
    json_extract_string(despacho_json, '$.encontroLegislativo.descricao')                    AS descricao_encontro,
    json_extract_string(despacho_json, '$.encontroLegislativo.casa')                         AS casa_encontro,
    CAST(json_extract_string(despacho_json, '$.encontroLegislativo.numero') AS INTEGER)      AS numero_encontro,
    json_extract_string(despacho_json, '$.encontroLegislativo.colegiado.casa')               AS colegiado_casa,
    CAST(json_extract_string(despacho_json, '$.encontroLegislativo.colegiado.codigo') AS BIGINT) AS colegiado_codigo,
    json_extract_string(despacho_json, '$.encontroLegislativo.colegiado.nome')               AS colegiado_nome,
    json_extract_string(despacho_json, '$.encontroLegislativo.colegiado.sigla')              AS colegiado_sigla,
    year_snapshot
  FROM desp
)
SELECT
  id_processo,
  id_despacho,
  id_encontro_legislativo,
  data_encontro,
  tipo_encontro,
  descricao_encontro,
  casa_encontro,
  numero_encontro,
  colegiado_casa,
  colegiado_codigo,
  colegiado_nome,
  colegiado_sigla,
  year_snapshot
FROM encontro
WHERE id_encontro_legislativo IS NOT NULL;


Running query in 'duck'

Count
4616


# 3. Regularização

# 3.1. Normalização de PKs

### 3.1.1. Bloco

In [126]:
%%sql
ALTER TABLE bloco_senado
    ALTER COLUMN codigo_bloco TYPE BIGINT;

ALTER TABLE bloco_senado
    ADD CONSTRAINT pk_bloco PRIMARY KEY (codigo_bloco);

Running query in 'duck'

Success


### 3.1.2. Parlamentar

In [127]:
%%sql
CREATE TABLE parlamentares_senado_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY codigo_parlamentar ORDER BY year_snapshot DESC) AS rn
    FROM parlamentar_senado
)
WHERE rn = 1;

DROP TABLE parlamentar_senado;
ALTER TABLE parlamentares_senado_dedup RENAME TO parlamentar_senado;


Running query in 'duck'

Success


In [128]:
%%sql
ALTER TABLE parlamentar_senado
    ALTER COLUMN codigo_parlamentar TYPE BIGINT;

ALTER TABLE parlamentar_senado
    ADD CONSTRAINT pk_parlamentar PRIMARY KEY (codigo_parlamentar);

Running query in 'duck'

Success


### 3.1.3. Partido

In [129]:
%%sql
ALTER TABLE partido_senado
    ALTER COLUMN codigo_partido TYPE BIGINT;

ALTER TABLE partido_senado
    ADD CONSTRAINT pk_partido PRIMARY KEY (codigo_partido);

Running query in 'duck'

Success


### 3.1.4. Processo

In [130]:
%%sql
ALTER TABLE processo_senado
    ALTER COLUMN id_processo TYPE BIGINT;

ALTER TABLE processo_senado
    ADD CONSTRAINT pk_processo PRIMARY KEY (id_processo);

Running query in 'duck'

Success


### 3.1.5. Emendas

In [131]:
%%sql
ALTER TABLE emendas_senado
    ALTER COLUMN id_emenda TYPE BIGINT;

ALTER TABLE emendas_senado
    ADD CONSTRAINT pk_emenda PRIMARY KEY (id_emenda);

Running query in 'duck'

Success


### 3.1.6. Relatorias

In [132]:
%%sql
ALTER TABLE relatorias_senado
    ALTER COLUMN id_relatoria TYPE BIGINT;

ALTER TABLE relatorias_senado
    ADD CONSTRAINT pk_relatoria PRIMARY KEY (id_relatoria);

Running query in 'duck'

Success


### 3.1.7. Votações

In [133]:
%%sql
ALTER TABLE votacoes_senado
    ALTER COLUMN id_votacao TYPE BIGINT;

ALTER TABLE votacoes_senado
    ADD CONSTRAINT pk_votacao PRIMARY KEY (id_votacao);

Running query in 'duck'

Success


### 3.1.8. Votos

In [134]:
%%sql
ALTER TABLE votos_senado
    ALTER COLUMN id_voto TYPE BIGINT;

ALTER TABLE votos_senado
    ADD CONSTRAINT pk_voto PRIMARY KEY (id_voto);

Running query in 'duck'

Success


### 3.1.9. Autoria-iniciativa

In [135]:
%%sql
ALTER TABLE autoria_iniciativa_senado
    ALTER COLUMN id_autoria_iniciativa TYPE BIGINT;

ALTER TABLE autoria_iniciativa_senado
    ADD CONSTRAINT pk_autoria_iniciativa PRIMARY KEY (id_autoria_iniciativa);

Running query in 'duck'

Success


### 3.1.10. Situações

In [136]:
%%sql
ALTER TABLE situacoes_senado
    ALTER COLUMN id_situacao TYPE BIGINT;

ALTER TABLE situacoes_senado
    ADD CONSTRAINT pk_situacao PRIMARY KEY (id_situacao);

Running query in 'duck'

Success


### 3.1.11. Despachos

In [137]:
%%sql
SELECT t.*
FROM despachos_senado t
JOIN (
    SELECT id_despacho
    FROM despachos_senado
    GROUP BY id_despacho
    HAVING COUNT(*) > 1
) dups
ON t.id_despacho = dups.id_despacho
ORDER BY t.id_despacho;

Running query in 'duck'

id_processo,id_despacho,data_despacho,cancelado,tipo_motivacao,sigla_tipo_motivacao,year_snapshot
7412455,7824068,2018-08-09,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7621491,7824068,2018-08-09,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693160,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693167,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693182,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693177,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693217,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693226,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7703389,7893781,2018-12-10,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7703387,7893781,2018-12-10,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018


In [138]:
%%sql
ALTER TABLE despachos_senado
    ALTER COLUMN id_despacho TYPE BIGINT;

ALTER TABLE despachos_senado
    ALTER COLUMN id_processo TYPE BIGINT;

ALTER TABLE despachos_senado
    ADD CONSTRAINT pk_despacho PRIMARY KEY (id_despacho, id_processo);

Running query in 'duck'

Success


### 3.1.12. Outros números

In [139]:
%%sql
ALTER TABLE outros_numeros_senado
    ALTER COLUMN id_outro_numero TYPE BIGINT;

ALTER TABLE outros_numeros_senado
    ADD CONSTRAINT pk_outro_numero PRIMARY KEY (id_outro_numero);

Running query in 'duck'

Success


### 3.1.13. Autoria do documento

In [140]:
%%sql
ALTER TABLE documento_autoria_senado
    ALTER COLUMN id_documento_autoria TYPE BIGINT;

ALTER TABLE documento_autoria_senado
    ADD CONSTRAINT pk_documento_autoria PRIMARY KEY (id_documento_autoria);

Running query in 'duck'

Success


### 3.1.14. Processos relacionados

In [141]:
%%sql
ALTER TABLE processos_relacionados_senado
    ALTER COLUMN id_processo_relacionado TYPE BIGINT;

ALTER TABLE processos_relacionados_senado
    ADD CONSTRAINT pk_processo_relacionado PRIMARY KEY (id_processo_relacionado);

Running query in 'duck'

Success


### 3.1.15. Autuações

In [142]:
%%sql
ALTER TABLE autuacoes_senado
    ALTER COLUMN autuacao_idx TYPE BIGINT;

ALTER TABLE autuacoes_senado
    ADD CONSTRAINT pk_autuacao PRIMARY KEY (id_processo, autuacao_idx);

Running query in 'duck'

Success


### 3.1.16. Informes legislativos

In [143]:
%%sql
ALTER TABLE informes_legislativos_senado
    ALTER COLUMN id_informe_legislativo TYPE BIGINT;

ALTER TABLE informes_legislativos_senado
    ADD CONSTRAINT pk_informe_legislativo PRIMARY KEY (id_informe_legislativo);

Running query in 'duck'

Success


### 3.1.17. Documentos associados a informes

In [144]:
%%sql
ALTER TABLE informes_documentos_associados_senado
    ALTER COLUMN id_documento_associado TYPE BIGINT;

ALTER TABLE informes_documentos_associados_senado
    ADD CONSTRAINT pk_documento_associado PRIMARY KEY (id_documento_associado);

Running query in 'duck'

Success


### 3.1.18. Providências

In [145]:
%%sql
ALTER TABLE providencias_senado
    ALTER COLUMN id_providencia TYPE BIGINT;

ALTER TABLE providencias_senado
    ADD CONSTRAINT pk_providencia PRIMARY KEY (
        id_processo,
        id_despacho,
        id_providencia
    );

Running query in 'duck'

Success


### 3.1.19. Movimentações

In [146]:
%%sql
ALTER TABLE movimentacoes_senado
    ADD CONSTRAINT pk_movimentacao PRIMARY KEY (
        id_processo,
        autuacao_idx,
        movimentacao_idx
    );

Running query in 'duck'

Success


### 3.1.20. Unidades destinatárias

In [147]:
%%sql
ALTER TABLE unidades_destinatarias_senado
    ALTER COLUMN id_unidade_destinataria TYPE BIGINT;

ALTER TABLE unidades_destinatarias_senado
    ADD CONSTRAINT pk_unidade_destinataria PRIMARY KEY (id_unidade_destinataria);

Running query in 'duck'

Success


### 3.1.21. Encontros legislativos

In [148]:
%%sql
SELECT t.*
FROM encontro_legislativo_senado t
JOIN (
    SELECT id_encontro_legislativo
    FROM encontro_legislativo_senado
    GROUP BY id_encontro_legislativo
    HAVING COUNT(*) > 1
) dups
ON t.id_encontro_legislativo = dups.id_encontro_legislativo
ORDER BY t.id_encontro_legislativo;

Running query in 'duck'

id_processo,id_despacho,id_encontro_legislativo,data_encontro,tipo_encontro,descricao_encontro,casa_encontro,numero_encontro,colegiado_casa,colegiado_codigo,colegiado_nome,colegiado_sigla,year_snapshot
7620082,7821741,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7620084,7821831,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7620086,7821866,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7620120,7821808,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7620163,7821850,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7620343,7822123,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7412505,7821457,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7621240,7823122,7620806,2018-08-08,SES,Sessão Deliberativa Ordinária,SF,111,SF,1998,Plenário do Senado Federal,PLEN,2018
7412485,7822751,7620806,2018-08-08,SES,Sessão Deliberativa Ordinária,SF,111,SF,1998,Plenário do Senado Federal,PLEN,2018
7621131,7823020,7620806,2018-08-08,SES,Sessão Deliberativa Ordinária,SF,111,SF,1998,Plenário do Senado Federal,PLEN,2018


In [149]:
%%sql
CREATE TABLE encontro_legislativo_senado_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY id_encontro_legislativo ORDER BY year_snapshot DESC) AS rn
    FROM encontro_legislativo_senado
)
WHERE rn = 1;

DROP TABLE encontro_legislativo_senado;
ALTER TABLE encontro_legislativo_senado_dedup RENAME TO encontro_legislativo_senado;


Running query in 'duck'

Success


In [150]:
%%sql
ALTER TABLE encontro_legislativo_senado
    ALTER COLUMN id_encontro_legislativo TYPE BIGINT;

ALTER TABLE encontro_legislativo_senado
    ADD CONSTRAINT pk_encontro_legislativo PRIMARY KEY (
        id_encontro_legislativo
    );

Running query in 'duck'

Success


## 3.2. Deduplicação de tabelas relacionais

### 3.2.1. Informes legislativos

In [151]:
%%sql
SELECT a.*
FROM informes_legislativos_senado a
JOIN (
    SELECT id_processo, id_informe
    FROM informes_legislativos_senado
    GROUP BY id_processo, id_informe
    HAVING COUNT(*) > 1
) dups
ON  a.id_processo = dups.id_processo
-- AND a.id_ente = dups.id_ente
-- AND a.autor = dups.autor
-- AND a.sigla_ente = dups.sigla_ente
AND a.id_informe = dups.id_informe
ORDER BY a.id_processo, a.id_informe


Running query in 'duck'

id_informe_legislativo,id_processo,id_informe,data_informe,descricao,id_situacao_iniciada,sigla_situacao_iniciada,ente_adm_casa,ente_adm_id,ente_adm_nome,ente_adm_sigla,colegiado_casa,colegiado_codigo,colegiado_nome,colegiado_sigla,year_snapshot
41209,500398,2119354,2023-03-08,"Aguardando inclusão em Ordem do Dia do Requerimento nº 127, de 2023, do Senador Veneziano Vital do Rêgo e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2017
41871,500398,2119354,2023-03-08,"Aguardando inclusão em Ordem do Dia do Requerimento nº 127, de 2023, do Senador Veneziano Vital do Rêgo e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2017
42504,500398,2119354,2023-03-08,"Aguardando inclusão em Ordem do Dia do Requerimento nº 127, de 2023, do Senador Veneziano Vital do Rêgo e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2017
12940,502077,2080636,2022-03-16,"Concluída a instrução das matérias, fica aberto o prazo de cinco dias úteis para recebimento de emendas, nos termos do art. 235, II, ""d"", do Regimento Interno.",88,AGREMESA,SF,13594,Secretaria de Atas e Diários,SEADI,SF,1998,Plenário do Senado Federal,PLEN,2016
10909,502077,2080636,2022-03-16,"Concluída a instrução das matérias, fica aberto o prazo de cinco dias úteis para recebimento de emendas, nos termos do art. 235, II, ""d"", do Regimento Interno.",88,AGREMESA,SF,13594,Secretaria de Atas e Diários,SEADI,SF,1998,Plenário do Senado Federal,PLEN,2016
20249,504573,2119092,2023-03-03,"Aguardando inclusão em Ordem do Dia do Requerimento nº 102, de 2023, da Senadora Professora Dorinha Seabra e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2015
22020,504573,2119092,2023-03-03,"Aguardando inclusão em Ordem do Dia do Requerimento nº 102, de 2023, da Senadora Professora Dorinha Seabra e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2015
23716,504573,2119092,2023-03-03,"Aguardando inclusão em Ordem do Dia do Requerimento nº 102, de 2023, da Senadora Professora Dorinha Seabra e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2015
25308,504573,2119092,2023-03-03,"Aguardando inclusão em Ordem do Dia do Requerimento nº 102, de 2023, da Senadora Professora Dorinha Seabra e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2015
26789,504573,2119092,2023-03-03,"Aguardando inclusão em Ordem do Dia do Requerimento nº 102, de 2023, da Senadora Professora Dorinha Seabra e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2015


In [152]:
%%sql
SELECT SUM(c) AS total_duplicadas
FROM (
    SELECT COUNT(*) - 1 AS c
    FROM informes_legislativos_senado
    GROUP BY id_processo, id_informe, descricao, data_informe, sigla_situacao_iniciada, data_informe
    HAVING COUNT(*) > 1
) t;

Running query in 'duck'

total_duplicadas
158784


In [153]:
%%sql
DELETE FROM informes_legislativos_senado
WHERE rowid IN (
    SELECT rowid
    FROM (
        SELECT 
            rowid,
            ROW_NUMBER() OVER (
                PARTITION BY id_processo, id_informe, descricao, data_informe, sigla_situacao_iniciada, data_informe
                ORDER BY rowid
            ) AS rn
        FROM informes_legislativos_senado
    ) t
    WHERE rn > 1
);

Running query in 'duck'

Count
158784


# 4. Limpeza

In [ ]:
conn.close()

: 